# Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    EarlyStoppingCallback
)
from transformers.trainer_utils import get_last_checkpoint
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate
import wandb
from datetime import datetime
import time
from tqdm.auto import tqdm
import sqlite3
import sqlparse
import _config

import os
import psutil
import GPUtil
import gc


# Set the verbosity to WARNING to suppress INFO messages
evaluate.logging.set_verbosity_warning()

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["WANDB_API_KEY"] = _config.WANDB_API_KEY
os.environ["WANDB_PROJECT"] = _config.WANDB_PROJECT

ENABLE_THINKING = False

2026-01-27 09:00:59.917388: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2026-01-27 09:01:18] INFO _client.py:1025: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[2026-01-27 09:01:18] INFO _client.py:1025: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[2026-01-27 09:01:18] INFO _client.py:1025: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[2026-01-27 09:01:19] INFO _client.py:1025: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[2026-01-27 09:01:19] INFO _client.py:1025: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[2026-01-27 09:01:19] INFO _client.py:1025: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK

# Utils

In [2]:
def get_vm_usage_metrics():
    # CPU usage
    cpu_load = psutil.cpu_percent(interval=1, percpu=True)
    for id, load in enumerate(cpu_load):
        print(f"CPU {id} load: {load:.2f}")
    # RAM usage
    ram = psutil.virtual_memory()
    print(f"RAM Total: {ram.total/(1024**3):.2f} GB, Used: {(ram.used)/(1024**3):.2f} GB")
    # GPU
    if torch.cuda.is_available():
        gpus = GPUtil.getGPUs()
        for gpu in gpus:
            print(f"GPU {gpu.id} ({gpu.name}) load: {gpu.load*100}%")
            print(f"GPU {gpu.id} ({gpu.name}) VRAM Total: {gpu.memoryTotal} MB, Used {gpu.memoryUsed} MB")
    # Disk 
    disk = psutil.disk_usage('/')
    print(f"Disk Total: {disk.total/(1024**3):.2f} GB, Used: {(disk.used)/(1024**3):.2f} GB")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Device: {device}')
get_vm_usage_metrics()

Device: cuda
CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 1.00
CPU 3 load: 0.00
RAM Total: 27.40 GB, Used: 1.85 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 3.0 MB
Disk Total: 60.95 GB, Used: 51.38 GB


In [3]:
def generate_model_response_batch(model, tokenizer, messages_list, enable_thinking=False, max_new_tokens=512):
    texts = [
        tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=enable_thinking
        )
        for messages in messages_list
    ]

    model_inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        padding_side='left'
    ).to(model.device)

    model.eval()
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens
    )

    responses = []
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids):
        # Slice to get only generated part
        output_only_ids = output_ids[len(input_ids):].tolist()

        # Try to find `</think>` (id 151668)
        try:
            index = len(output_only_ids) - output_only_ids[::-1].index(151668)
        except ValueError:
            index = 0

        if enable_thinking:
            thinking_content = tokenizer.decode(
                output_only_ids[:index],
                skip_special_tokens=True
            ).strip("\n")
            content = tokenizer.decode(
                output_only_ids[index:],
                skip_special_tokens=True
            ).strip("\n")
        else:
            thinking_content = None
            content = tokenizer.decode(
                output_only_ids,
                skip_special_tokens=True
            ).strip("\n")

        responses.append({
            'thinking_content': thinking_content,
            'content': content
        })

    return responses

# Data

In [4]:
ds = datasets.load_dataset('gretelai/synthetic_text_to_sql', streaming=False)
ds_train, ds_test = ds['train'], ds['test']

ds_train_subset = ds_train.train_test_split(test_size=0.2, seed=42)['test']
split = ds_train_subset.train_test_split(test_size=0.025, seed=42)
ds_train = split['train']
ds_valid = split['test']

ds_train

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 19500
})

In [5]:
# datasets for GRPO must include a column "prompt"
def construct_message(example):
    return {"prompt": [
        {"role": "system", "content": f"The user asks a question. Your task is to generate the SQL query to answer that question. Return SQL query only. The context of the question is the following: '{example['sql_context']}'"},
        {"role": "user", "content": example['sql_prompt']}
    ]}
ds_train = ds_train.map(construct_message)
ds_valid = ds_valid.map(construct_message)

# rename the ground_truth column
ds_train = ds_train.rename_column("sql", "ground_truth")
ds_valid = ds_valid.rename_column("sql", "ground_truth")

ds_train

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'ground_truth', 'sql_explanation', 'prompt'],
    num_rows: 19500
})

# Reward function

In [6]:
rouge = evaluate.load("rouge")

def normalize_sql(sql):
    return sqlparse.format(sql, reindent=True, keyword_case='upper').strip()

def compute_rouge(reference, prediction):
    result = rouge.compute(predictions=[prediction], references=[reference])
    return result['rougeL']


def evaluate_sql_response(reference, prediction, sql_context):
    # ROUGE-L
    rouge_score = compute_rouge(reference, prediction)

    for ref, pred,  in zip(reference, prediction):
        # execution check
        try:
            conn = sqlite3.connect(":memory:")
            cursor = conn.cursor()
            
            cursor.executescript(sql_context)
            cursor.execute(reference)
            ref_result = cursor.fetchall()
            
            cursor.execute(prediction)
            model_result = cursor.fetchall()
            
            execution_match = ref_result == model_result
        except Exception:
            execution_match = False
        finally:
            conn.close()
        
        # final score
        if execution_match:
            final_score = 1.0
        else:
            final_score = 0.7 * rouge_score

    return {
        "rougeL": round(rouge_score, 4),
        "execution_match": execution_match,
        "final_score": final_score
    }


def reward_func(completions, ground_truth, sql_context, **kwargs):
    scores = [
        evaluate_sql_response(
            reference=reference,
            prediction=prediction[0]["content"],
            sql_context=context
        )["final_score"]
        for reference, prediction, context in zip(ground_truth, completions, sql_context)
    ]
    return scores

# Model

In [7]:
checkpoint = "Qwen/Qwen3-0.6B"
model_path = "./qlora-final_model_all_linear_r64-output/final/"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    # attn_implementation="sdpa",
    device_map="auto",
    dtype=torch.float32,
    # quantization_config=bnb_config
)

# Manually set LoRA parameters as trainable
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True


model.config.use_cache = False
model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)
model.enable_input_require_grads()

get_vm_usage_metrics()

[2026-01-27 09:01:34] INFO modeling.py:1004: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 1.00
CPU 3 load: 0.00
RAM Total: 27.40 GB, Used: 2.08 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 3371.0 MB
Disk Total: 60.95 GB, Used: 51.38 GB


# GRPO

In [ ]:
torch.cuda.empty_cache()

# timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
timestamp = '2026-01-25_09-24-49'
RUN_NAME = f'sft-grpo-lora-lr1e6-ngen4-epochs1-{timestamp}'
OUTPUT_DIR = './sft-grpo-lr1e6-ngen4-output'
RESUME_TRAINING = True

PER_DEVICE_BATCH_SIZE = 8
effective_batch_size = 16
epochs=1
learning_rate = 1e-6 # changed from 1e-5
warmup_ratio = 0.1
lora_r = 16*4
lora_alpha = 64*4
lora_dropout = 0.01

gradient_accumulation_steps = int(effective_batch_size / PER_DEVICE_BATCH_SIZE)

wandb.init(
    project=os.environ["WANDB_PROJECT"],
    name=RUN_NAME,
    id='yrm8qwl9' ,         # resume previous run if available
    resume="allow",    # allows resuming crashed run
)



training_args = GRPOConfig(
    output_dir=OUTPUT_DIR,
    
    chat_template_kwargs = {"enable_thinking": False},
    num_train_epochs=epochs,
    num_generations=4,
    # use_liger_kernel=True,
    
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    lr_scheduler_type="cosine",
    warmup_ratio=warmup_ratio,
    save_strategy="steps",
    save_steps=30,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=30,
    per_device_eval_batch_size=PER_DEVICE_BATCH_SIZE*4,
    eval_accumulation_steps=1,
    # eval_kwargs={"num_generations": 1},
    num_generations_eval=1,
    logging_strategy="steps",
    logging_steps=30,
    report_to=['wandb'],
    run_name=RUN_NAME,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=1,
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # generate_during_eval=True
)

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules='all-linear'
)
# model.requires_grad_(False)                     # freeze base weights (precautionary)
model_peft = get_peft_model(model, peft_config) # inject a LoRA adapter

trainer = GRPOTrainer(
    processing_class=tokenizer,
    model=model_peft,
    args=training_args,
    reward_funcs=[reward_func],
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


# Training setup summary
dataset_size = len(ds_train)
steps_per_epoch = dataset_size // (PER_DEVICE_BATCH_SIZE * gradient_accumulation_steps)
total_steps = steps_per_epoch * epochs
warmup_steps = int(total_steps * warmup_ratio)

print("===== Training Setup Summary =====")
print(f"Num epochs:            {epochs}")
print(f"Effective batch size:  {effective_batch_size}")
print(f"Per-device batch size: {PER_DEVICE_BATCH_SIZE}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Dataset size:          {dataset_size}")
print(f"Steps per epoch:       {steps_per_epoch}")
print(f"Total training steps:  {total_steps}")
print(f"Warmup steps:          {warmup_steps}")
print(f"Logging steps:         {training_args.logging_steps}")
print("===================================")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# Training
last_checkpoint = None
if RESUME_TRAINING and os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint is not None:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting fresh training run")
    trainer.train()

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: olialeshka (olialeshka-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
/home/olialeshka_1/myenv/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/olialeshka_1/myenv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
[2026-01-27 09:02:19] INFO spawn.py:77: x86_64-linux-gnu-gcc -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -fPIC -c /tmp/tmpdbwy_1wc/test.c -o /tmp/tmpdbwy_1wc/test.o
[2026-

===== Training Setup Summary =====
Num epochs:            1
Effective batch size:  16
Per-device batch size: 8
Gradient accumulation: 2
Dataset size:          19500
Steps per epoch:       1218
Total training steps:  1218
Warmup steps:          121
Logging steps:         30
Start time: 2026-01-27_09-02-20
Resuming training from checkpoint: ./sft-grpo-lr1e6-ngen4-output/checkpoint-4020


[2026-01-27 09:02:39] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:39] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:40] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:41] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:41] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:41] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:41] INFO rouge_scorer.py:83: Using default tok

Step,Training Loss,Validation Loss


[2026-01-27 09:02:50] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:51] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:52] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:52] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:52] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:52] INFO rouge_scorer.py:83: Using default tokenizer.
[2026-01-27 09:02:52] INFO rouge_scorer.py:83: Using default tok

In [ ]:
# model.save_pretrained(f"{OUTPUT_DIR}/best_model")

# Test

In [ ]:
OUTPUT_DIR = './sft-grpo-lr1e6-ngen4-output'
checkpoint = f"{OUTPUT_DIR}/checkpoint-4875/"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, dtype=torch.float16).to(device)
model.eval()

ds = datasets.load_dataset('gretelai/synthetic_text_to_sql', streaming=False)
ds_train, ds_test = ds['train'], ds['test']


def construct_message(prompt, context):
    return [
        {"role": "system", "content": f"The user asks a question. Your task is to generate the SQL query to answer that question. Return SQL query only. The context of the question is the following: '{context}'"},
        {"role": "user", "content": prompt}
    ]

def generate_model_response_batch(messages_list, enable_thinking=True, max_new_tokens=512):
    texts = [
        tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=enable_thinking
        )
        for messages in messages_list
    ]

    model_inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        padding_side='left'
    ).to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens
    )

    responses = []
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids):
        # Slice to get only generated part
        output_only_ids = output_ids[len(input_ids):].tolist()

        # Try to find `</think>` (id 151668)
        try:
            index = len(output_only_ids) - output_only_ids[::-1].index(151668)
        except ValueError:
            index = 0

        if enable_thinking:
            thinking_content = tokenizer.decode(
                output_only_ids[:index],
                skip_special_tokens=True
            ).strip("\n")
            content = tokenizer.decode(
                output_only_ids[index:],
                skip_special_tokens=True
            ).strip("\n")
        else:
            thinking_content = None
            content = tokenizer.decode(
                output_only_ids,
                skip_special_tokens=True
            ).strip("\n")

        responses.append({
            'thinking_content': thinking_content,
            'content': content
        })

    return responses


rouge = evaluate.load("rouge")

def normalize_sql(sql):
    return sqlparse.format(sql, reindent=True, keyword_case='upper').strip()

def compute_rouge(reference, prediction):
    result = rouge.compute(predictions=[prediction], references=[reference])
    return result['rougeL']

def evaluate_sql_response(reference, prediction, sql_context):
    # ROUGE-L
    rouge_score = compute_rouge(reference, prediction)
    
    # execution check
    try:
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()
        
        cursor.executescript(sql_context)
        cursor.execute(reference)
        ref_result = cursor.fetchall()
        
        cursor.execute(prediction)
        model_result = cursor.fetchall()
        
        execution_match = ref_result == model_result
    except Exception:
        execution_match = False
    finally:
        conn.close()
    
    # final score
    if execution_match:
        final_score = 1.0
    else:
        final_score = 0.7 * rouge_score

    return {
        "rougeL": round(rouge_score, 4),
        "execution_match": execution_match,
        "final_score": final_score
    }

In [ ]:
BATCH_SIZE = 32
ENABLE_THINKING = False
MAX_NEW_TOKENS = 512


prompts = [ds_test[id]['sql_prompt'] for id in range(len(ds_test))]
contexts = [ds_test[id]['sql_context'] for id in range(len(ds_test))]

responses = []
print(f"Start time: {time.ctime(time.time())}")
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch_prompts = prompts[i : i + BATCH_SIZE]
    batch_contexts = contexts[i : i + BATCH_SIZE]

    messages_list = [
        construct_message(prompt=p, context=c)
        for p, c in zip(batch_prompts, batch_contexts)
    ]

    batch_responses = generate_model_response_batch(messages_list, enable_thinking=ENABLE_THINKING, max_new_tokens=MAX_NEW_TOKENS)

    responses.extend(batch_responses)

print(f"End time: {time.ctime(time.time())}")

In [ ]:
references = [ds_test[id]['sql'] for id in range(len(ds_test))]
predictions = [responses[id]['content'] for id in range(len(ds_test))]

scores = [
    evaluate_sql_response(
        reference=reference,
        prediction=prediction,
        sql_context=context
    )
    for reference, prediction, context in tqdm(zip(references, predictions, contexts), total=len(ds_test))
]

In [13]:
print(f"Mean test set score: {np.mean([score['final_score'] for score in scores]):.3f}")

Mean test set score: 0.756
